In [45]:
import pandas as pd 
import xml.etree.ElementTree as ET
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm

In [ ]:

CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver'
INPUT_FILE = 'players_list_xml_foa.xml'
OUTPUT_FILE = 'updated_titled_otb_df.csv'

In [34]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)     
pd.set_option('display.expand_frame_repr', False)  

In [2]:
file_path = INPUT_FILE

tree = ET.parse(file_path)
root = tree.getroot()

data = []
for player in root.findall('player'):
    player_data = {
        "fideid": player.find("fideid").text,
        "name": player.find("name").text,
        "country": player.find("country").text,
        "sex": player.find("sex").text,
        "title": player.find("title").text,
        "w_title": player.find("w_title").text,
        "o_title": player.find("o_title").text,
        "foa_title": player.find("foa_title").text,
        "rating": player.find("rating").text,
        "games": player.find("games").text,
        "k": player.find("k").text,
        "rapid_rating": player.find("rapid_rating").text,
        "rapid_games": player.find("rapid_games").text,
        "rapid_k": player.find("rapid_k").text,
        "blitz_rating": player.find("blitz_rating").text,
        "blitz_games": player.find("blitz_games").text,
        "blitz_k": player.find("blitz_k").text,
        "birthday": player.find("birthday").text,
        "flag": player.find("flag").text if player.find("flag") is not None else None,
    }
    data.append(player_data)

df = pd.DataFrame(data)

In [3]:
df.to_csv("fide_players.csv", index=False)

In [4]:
df = pd.read_csv("fide_players.csv")

In [5]:
df.head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,k,rapid_rating,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag
0,167105475,"-, -",FID,M,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,1999.0,NaN
1,10292519,"A A M Imtiaz, Chowdhury",BAN,M,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,1975.0,NaN
2,10688862,"A Abdel Maabod, Hoda",EGY,F,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,2009.0,w
3,33496722,A Aman,IND,M,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,1996.0,NaN
4,537001345,A Arbhin Vanniarajan,IND,M,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,2018.0,NaN


In [6]:
df.shape

(1561917, 19)

In [7]:
titled_otb_df = df[
    (df['title'].notna()) |
    (df['w_title'].notna())
]

In [8]:
titled_otb_df.shape

(22557, 19)

In [10]:
titled_otb_df = titled_otb_df.copy()
unique_titles = titled_otb_df['title'].dropna().unique().tolist()
unique_w_titles = titled_otb_df['w_title'].dropna().unique().tolist()

all_unique_titles = unique_titles + unique_w_titles

titled_otb_df['is_scraped'] = False

for title in all_unique_titles:
    titled_otb_df.loc[:, title] = titled_otb_df['title'].eq(title) | titled_otb_df['w_title'].eq(title)
    
    titled_otb_df.loc[:, f"{title}_year"] = None  

In [22]:
def init_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    service = Service(CHROMEDRIVER_PATH)
    return webdriver.Chrome(service=service, options=chrome_options)

def scrape_player_data(driver, fide_ids):
    data = []
    for fide_id in tqdm(fide_ids, desc="Scraping Players [Total]"):
        titles = {}
        game_results = {
            "white_results": {"win": 0, "draw": 0, "loss": 0},
            "black_results": {"win": 0, "draw": 0, "loss": 0},
        }
        try:
            profile_url = f"https://ratings.fide.com/profile/{fide_id}"
            driver.get(profile_url)
            titles_table = driver.find_element(By.XPATH, "//th[text()='Titles']/ancestor::table")
            titles_rows = titles_table.find_elements(By.XPATH, ".//tr")[1:]
            for row in titles_rows:
                columns = row.find_elements(By.TAG_NAME, "td")
                if len(columns) == 2:
                    title = columns[0].text.strip()
                    year = columns[1].text.strip()
                    titles[title] = year
        except Exception:
            pass 
        
        try:
            stats_url = f"https://ratings.fide.com/profile/{fide_id}/statistics"
            driver.get(stats_url)
            driver.implicitly_wait(10)
            charts_data = driver.execute_script("""
                var charts = Chart.instances;
                return [0, 1].map(index => {
                    var chart = charts[index];
                    return {
                        labels: chart.data.labels,
                        datasets: chart.data.datasets.map(dataset => dataset.data)
                    };
                });
            """)
            white_stats = charts_data[0]
            black_stats = charts_data[1]
            
            for label, value in zip(white_stats["labels"], white_stats["datasets"][0]):
                if "Win" in label:
                    game_results["white_results"]["win"] = int(value)
                elif "Draw" in label:
                    game_results["white_results"]["draw"] = int(value)
                elif "Loss" in label:
                    game_results["white_results"]["loss"] = int(value)
            
            for label, value in zip(black_stats["labels"], black_stats["datasets"][0]):
                if "Win" in label:
                    game_results["black_results"]["win"] = int(value)
                elif "Draw" in label:
                    game_results["black_results"]["draw"] = int(value)
                elif "Loss" in label:
                    game_results["black_results"]["loss"] = int(value)
        except Exception:
            pass  
        
        data.append({"fide_id": fide_id, "titles": titles, "game_results": game_results})
    return data

def process_scraped_data(df, scraped_data):
    for record in scraped_data:
        fide_id = record["fide_id"]
        titles = record["titles"]
        game_results = record["game_results"]
        
        for title, year in titles.items():
            title_col = f"{title.replace(' ', '_')}_year"
            if title_col not in df.columns:
                df[title_col] = None
            df.loc[df["fideid"] == fide_id, title_col] = year
        
        df.loc[df["fideid"] == fide_id, "white_win"] = game_results["white_results"]["win"]
        df.loc[df["fideid"] == fide_id, "white_draw"] = game_results["white_results"]["draw"]
        df.loc[df["fideid"] == fide_id, "white_loss"] = game_results["white_results"]["loss"]
        df.loc[df["fideid"] == fide_id, "black_win"] = game_results["black_results"]["win"]
        df.loc[df["fideid"] == fide_id, "black_draw"] = game_results["black_results"]["draw"]
        df.loc[df["fideid"] == fide_id, "black_loss"] = game_results["black_results"]["loss"]
        df.loc[df["fideid"] == fide_id, "is_scraped"] = True

def start_scraping_process(df, start_row=0):
    total_rows = len(df)
    skipped_rows = len(df[df["is_scraped"] == True])
    unscripted_rows = len(df[df["is_scraped"] == False])
    
    print(f"Total rows: {total_rows}")
    print(f"Skipped rows (already scraped): {skipped_rows}")
    print(f"Unscraped rows: {unscripted_rows}")
    
    driver = init_driver()
    try:
        rows_to_scrape = df[(df.index >= start_row) & (df["is_scraped"] == False)]
        fide_ids = rows_to_scrape["fideid"].tolist()
        
        total_records = len(fide_ids)
        with tqdm(total=total_records, desc="Scraping Players [Batch]") as pbar:
            batch_size = 50
            for i in range(0, len(fide_ids), batch_size):
                batch_fide_ids = fide_ids[i:i + batch_size]
                scraped_data = scrape_player_data(driver, batch_fide_ids)
                process_scraped_data(df, scraped_data)
                
                pbar.update(len(batch_fide_ids))
    finally:
        driver.quit()
    


In [24]:
start_scraping_process(titled_otb_df)

Total rows: 22557
Skipped rows (already scraped): 22557
Unscraped rows: 0


Scraping Players [Batch]: 0it [00:00, ?it/s]


Scraping process completed. Updated data saved to updated_titled_otb_df.csv


In [27]:
titled_otb_df[titled_otb_df["is_scraped"] == True].shape

(22557, 52)

In [28]:
titled_otb_df[titled_otb_df["is_scraped"] == False].shape

(0, 52)

In [29]:
titled_otb_df

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,k,rapid_rating,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag,is_scraped,IM,IM_year,WFM,WFM_year,WCM,WCM_year,GM,GM_year,WIM,WIM_year,FM,FM_year,CM,CM_year,WGM,WGM_year,WH,WH_year,International_Master_(IM)_year,white_win,white_draw,white_loss,black_win,black_draw,black_loss,FIDE_Master_(FM)_year,Candidate_Master_(CM)_year,Woman_FIDE_Master_(WFM)_year,Woman_Candidate_Master_(WCM)_year,Grandmaster_(GM)_year,Woman_International_Master_(WIM)_year,Woman_Grandmaster_(WGM)_year
169,1701991,"Aaberg, Anton",SWE,M,IM,NaN,NaN,NaN,2322,1,10,2331,0,20,0,0,0,1972.0,NaN,True,True,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,2013,37.0,33.0,22.0,39.0,37.0,21.0,None,None,None,None,None,None,None
199,1407589,"Aabling-Thomsen, Jakob",DEN,M,IM,NaN,NaN,NaN,2327,0,10,0,0,0,0,0,0,1985.0,NaN,True,True,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,2016,187.0,83.0,92.0,147.0,81.0,142.0,2009,None,None,None,None,None,None
591,25678191,Aaditya Dhingra,IND,M,IM,NaN,NaN,NaN,2421,16,10,2208,8,20,2268,0,20,2006.0,NaN,True,True,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,2023,0.0,0.0,0.0,0.0,0.0,0.0,None,2020,None,None,None,None,None
748,25778293,Aadya Gupta,IND,F,WFM,WFM,NaN,NaN,1968,9,40,1748,0,40,1888,0,40,2010.0,w,True,False,None,True,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,None,0.0,0.0,0.0,0.0,0.0,0.0,None,None,2024,None,None,None,None
760,25991426,Aadya Ranganath,IND,F,WCM,WCM,NaN,NaN,1900,18,40,1704,0,40,1747,9,40,2013.0,w,True,False,None,False,None,True,None,False,None,False,None,False,None,False,None,False,None,False,None,None,0.0,0.0,0.0,0.0,0.0,0.0,None,None,None,2025,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561688,24150789,"Zykina, Nadezhda",RUS,F,WFM,WFM,NaN,NaN,2148,0,20,2050,0,20,2055,0,20,1956.0,wi,True,False,None,True,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,None,0.0,0.0,0.0,0.0,0.0,0.0,None,None,None,None,None,None,None
1561734,1102338,"Zyla, Janusz",POL,M,FM,NaN,NaN,NaN,2181,0,20,2123,0,20,2182,0,20,1956.0,i,True,False,None,False,None,False,None,False,None,False,None,True,None,False,None,False,None,False,None,None,0.0,0.0,0.0,0.0,0.0,0.0,None,None,None,None,None,None,None
1561762,1189654,"Zylka, Stanislaw",POL,M,IM,NaN,NaN,NaN,2357,1,10,2281,0,20,2291,0,20,1999.0,NaN,True,True,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,False,None,None,0.0,0.0,0.0,0.0,0.0,0.0,None,None,None,None,None,None,None
1561777,4700295,"Zymberi, Astrit",KOS,M,FM,NaN,FI,NaN,2059,0,20,2117,0,20,0,0,0,1974.0,NaN,True,False,None,False,None,False,None,False,None,False,None,True,None,False,None,False,None,False,None,None,0.0,0.0,0.0,0.0,0.0,0.0,None,None,None,None,None,None,None


In [40]:
column_order = [
    'fideid', 'name', 'country', 'sex', 'title', 'w_title', 'o_title', 'foa_title',
    'rating', 'games', 'k', 'rapid_rating', 'rapid_games', 'rapid_k', 'blitz_rating',
    'blitz_games', 'blitz_k', 'birthday', 'flag', 'is_scraped', 'IM', 'IM_year',
    'WFM', 'WFM_year', 'WCM', 'WCM_year', 'GM', 'GM_year', 'WIM', 'WIM_year',
    'FM', 'FM_year', 'CM', 'CM_year', 'WGM', 'WGM_year', 'WH', 'WH_year',
    'International_Master_(IM)_year', 'white_win', 'white_draw', 'white_loss',
    'black_win', 'black_draw', 'black_loss', 'FIDE_Master_(FM)_year',
    'Candidate_Master_(CM)_year', 'Woman_FIDE_Master_(WFM)_year',
    'Woman_Candidate_Master_(WCM)_year', 'Grandmaster_(GM)_year',
    'Woman_International_Master_(WIM)_year', 'Woman_Grandmaster_(WGM)_year'
]

reordered_df = titled_otb_df[column_order].copy()



In [41]:
reordered_df.to_csv(OUTPUT_FILE, index=False)